# Improve Your Model Performance using Cross Validation

If you run your machine learning model several times, even with the same configuration, you may notice that your model performance may go up and down. Why would that happen? Since machine learning models try to approximate the data - there is always __uncertainty__ in there.

We want to limit the __uncertainty__ in our models, so that _the model can produce consistent results on unseen data_. In other words, if the model uncertainty is too high, the model may produce unreliable results.

## Why do models lose stability?
Let’s understand this using the below snapshot illustrating the fit of various models:

![Model Stability](https://www.analyticsvidhya.com/wp-content/uploads/2015/11/15.png)

Here, we are trying to find the relationship between size and price. To achieve this, we have taken the following steps:

1. In the first plot, you can observe high error (model fitting loosely to the data) - it is an example of “Underfitting”.
The first plot has a high error from training data points.
2. In the second plot, we just found the right relationship between price and size, i.e., low training error and generalization of the relationship.
3. In the third plot, we found a relationship which has almost zero training error. This is because the relationship is developed by considering each deviation in the data point (including noise), i.e., the model is too sensitive and captures random patterns which are present only in the current dataset. This is an example of “Overfitting”.

A common practice in data science competitions is to iterate over various models to find a better performing model. However, it becomes difficult to distinguish whether this improvement in score is coming because we are capturing the relationship better, or we are just over-fitting the data. To find the right answer for this question, we use validation techniques. This method helps us in achieving more generalized relationships.



## Before Cross Validation

So far, in all but one tutorials in this class, we have been using the traditional train-test split method for validation purposes. This method is called the __(_fixed_) hold-out method__. In this method, a fixed portion of the data (e.g. _20%_) is reserved for evaluation purposes.

Refresh your memory with the code below.

In [ ]:
# import the required packages
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import *
import pandas as pd
import numpy as np


from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# load the wine dataset
my_data = load_breast_cancer()
my_data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [ ]:
my_df = pd.DataFrame(my_data.data, columns=my_data.feature_names)
my_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
my_df['label'] = my_data.target
my_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [ ]:
# split the data into train/test
# test takes up 20% of the data
X_train, X_test,y_train, y_test\
    = train_test_split(my_data.data, my_data.target, test_size=0.2, random_state=2020)

In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((455, 30), (455,), (114, 30), (114,))

We discussed sometimes we might want to reserve a portion of the data for model optimization purposes - that portion of the data is called the validation set. So that method is called the _three-way hold-out method_.

![three way hold out](https://i.stack.imgur.com/pXAfX.png)

We can also do that with `train_test_split()`. Say we want to reserve `20%` for validation.

In [ ]:
X_train, X_val, y_train, y_val\
   = train_test_split(X_train, y_train, test_size=0.25, random_state=2020) # 0.25 x 0.8 = 0.2

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((341, 30), (114, 30), (341,), (114,))

Now we can try to fit the model multiple times to observe the variance in model performances with the __hold out method__.

In [ ]:
clf = SVC(C=1.0)
for i in range(10):
  fit = clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print('Accruracy for ', i, 'th round training: ', round(accuracy_score(y_test, y_pred), 4)) # no variance

Accruracy for  0 th round training:  0.8947
Accruracy for  1 th round training:  0.8947
Accruracy for  2 th round training:  0.8947
Accruracy for  3 th round training:  0.8947
Accruracy for  4 th round training:  0.8947
Accruracy for  5 th round training:  0.8947
Accruracy for  6 th round training:  0.8947
Accruracy for  7 th round training:  0.8947
Accruracy for  8 th round training:  0.8947
Accruracy for  9 th round training:  0.8947


We can observe from above there are no variances in the results - since `sklearn` will always optimize your model within the current configuration.

If we want to see some variance, we need to have different training/test sets (with the same 80:20 split). We can do that via the __Repeated Holdout__ method. See the code below.

In [ ]:
for i in range(10):
  # remove `random_state` so we have different training/test sample in every iteration
  X_train, X_test,y_train, y_test\
    = train_test_split(my_data.data, my_data.target, test_size=0.2)
  fit = clf.fit(X_train, y_train)
  y_pred = clf.predict(X_test)
  print('Accruracy for ', i, 'th round training: ', round(accuracy_score(y_test, y_pred), 4)) # some variance

Accruracy for  0 th round training:  0.9298
Accruracy for  1 th round training:  0.886
Accruracy for  2 th round training:  0.8772
Accruracy for  3 th round training:  0.9386
Accruracy for  4 th round training:  0.9123
Accruracy for  5 th round training:  0.8947
Accruracy for  6 th round training:  0.9035
Accruracy for  7 th round training:  0.9123
Accruracy for  8 th round training:  0.9123
Accruracy for  9 th round training:  0.886


Now you can see some bumping ups and downs in the results (variance).

So what is wrong with the hold out and the repeated holdout methods? Since you essentially is training your model in one shot, you may get a "lucky draw' of your data (in which your model outperforms the actual), or even worse, an "unlucky draw" (in which your model underperforms the actual). We do not want either situation - we want a __fair estimate__ of the model performance.

In other words, you want your model to be exposed to as much data as you can, so the model can learn a comprehensive pattern (not a partial image) from your data. Since we cannot use all the data for training, that is why we need __Cross Validation__.

## What is Cross Validation?
Cross Validation is a technique which involves reserving a particular sample of a dataset on which you do not train the model. Later, you test your model on this sample before finalizing it.

Here are the steps involved in k-fold cross validation:

1. Split your dataset into K (roughly) equal folds, and reserve 1 fold for evaluation/optimization purposes - note these two are related but different;
2. Train the model using the remaining (K-1) folds and the reserve sample as the test (validation) set. This will help you in gauging the effectiveness of your model’s performance.

If your model delivers a positive result on validation data, go ahead with the current model.

Even though k-fold cross validation is the most popular type, do not assume that it is the _only_ cross validation method.

## Common Methods for Cross Validation

Cross Validation (CV) is a family of sampling/model evaluation/model optimization methods. In the stats context, it is a sampling method. In the machine learning context, it is widely used for model evaluation and/or model optimization purposes.

There is a recommendation that every model needs to go through CV once, either for model evaluation or model optimization purposes.

Here is a list of common CV methods:
- Leave-One-Out Cross Validation (LOOCV) (_most extreme_)
- K-fold Cross Validation (_most popular_)
- Repeated K-fold Cross Validation
- Stratified K-fold Cross Validation (_best for imblanced data_)
- Cross Validation for Time Series (_fairly popular right now_)

Let's see how to implement them one by one.

### Leave-One-Out Cross Validation (LOOCV)

LOOCV is the most extreme CV method. In every iteration, only __one data point__ is used for testing, the remainder of the data is used for training.

Pros:
- Model is fit to almost the whole dataset; very little chance of having a "lucky/unlucky" draw;

Cons:
- Training is slow;
- Variance in model performances is high.

Even though `sklearn` has its own `LeaveOneOut` method, we can essentially use the `KFold()` method for that.

In [ ]:
X = my_data.data

kf = KFold(n_splits=len(X)) # split the data

# look at the first 3 iterations
i = 0
for train_index, test_index in kf.split(X):
  print("Training data contains", len(train_index), "data points")
  print("TEST data contains", len(test_index), "data points")
  i += 1
  if i > 3:
    break


Training data contains 568 data points
TEST data contains 1 data points
Training data contains 568 data points
TEST data contains 1 data points
Training data contains 568 data points
TEST data contains 1 data points
Training data contains 568 data points
TEST data contains 1 data points


We can see in every iteration, the training dataset contains $569 - 1 = 568$ instances, and the test set contains $1$ data point.

Even though we can implement the LOOCV in `sklearn`, it is not well supported in it. So we will stop here and move on to the next method, K-fold CV.

### K-fold Cross Validation

This is the most popular method in the context of CV.

Pros:
- Most balanced method;
- Can be used for both model evaluation and optmization purposes

Cons:
- if dataset is too small and K is too large, model might underfit
- if k is too small, model may overfit

For __evaluation purposes__, we can simply use the `cross_val_score` method.
- it takes the model, features, target, and K as function parameters
- by default the returned value is the accuracy score (e.g. classification accuracy for our model)

Below code performs a 5-fold CV using the `SVC` model above on our data - you can see five different _accuracy_ scores.


In [ ]:
y = my_data.target
cross_val_score(clf, X, y, cv=5)

array([0.85087719, 0.89473684, 0.92982456, 0.94736842, 0.9380531 ])

In [ ]:
# change k to 3
cross_val_score(clf, X, y, cv=3)

array([0.85263158, 0.93157895, 0.94708995])

In [ ]:
# change k to 10
cross_val_score(clf, X, y, cv=10)

array([0.89473684, 0.84210526, 0.89473684, 0.92982456, 0.92982456,
       0.92982456, 0.94736842, 0.92982456, 0.92982456, 0.91071429])

If you want the final score of the model, usually we will use the __average__ across the `k` iterations.

In [ ]:
print('final model accuracy:', cross_val_score(clf, X, y, cv=10).mean())

final model accuracy: 0.9138784461152882


You can also test how much the variance is in the results, you can check the _standard deviation_ of the scores.

In [ ]:
print('model accuracy variance:', np.std(cross_val_score(clf, X, y, cv=10)))

model accuracy variance: 0.02878745403168186


You can also specify using different metrics. For instance, we may want to focus on the _f1-score_ or the _ROC/AUC_ metric. All supported scoring metrics are listed [here](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter).

We can do that using the code below.

In [ ]:
cross_val_score(clf, X, y, cv=10, scoring='f1')

array([0.92105263, 0.88311688, 0.91891892, 0.94736842, 0.94736842,
       0.94594595, 0.95890411, 0.94594595, 0.94444444, 0.93333333])

In [ ]:
cross_val_score(clf, X, y, cv=10, scoring='roc_auc')

array([0.96753247, 0.95324675, 0.98412698, 0.96957672, 0.98809524,
       0.98015873, 0.97883598, 0.96296296, 0.98412698, 0.99047619])

The `cross_val_score` method is a shortcut for model evaluation purposes. The regular method is `KFold` - if we want to use the K-fold CV for model optimization purposes, we can use `KFold`, or specific methods for hyperparameter tuning that we will see next week.

### Repeated K-fold Cross Validation

Repeated K-fold CV is basically conducting the K-Fold cross validation `i` times. See the comparison below.

In [ ]:
# create some synthetic data for illustration
X_data = np.random.randint(5, size=(9, 2))
X_data

array([[3, 2],
       [0, 0],
       [4, 2],
       [0, 0],
       [2, 3],
       [1, 3],
       [0, 1],
       [2, 3],
       [3, 0]])

Regular K-fold CV:

In [ ]:
kf = KFold(n_splits=3, random_state=2020, shuffle=True)
for train_index, test_index in kf.split(X_data):
      print("Train:")
      print(X_data[train_index])
      print("Test:")
      print(X_data[test_index])
      print('\n')

Train:
[[3 2]
 [0 0]
 [0 0]
 [1 3]
 [0 1]
 [3 0]]
Test:
[[4 2]
 [2 3]
 [2 3]]


Train:
[[3 2]
 [4 2]
 [0 0]
 [2 3]
 [2 3]
 [3 0]]
Test:
[[0 0]
 [1 3]
 [0 1]]


Train:
[[0 0]
 [4 2]
 [2 3]
 [1 3]
 [0 1]
 [2 3]]
Test:
[[3 2]
 [0 0]
 [3 0]]




Repeated K-fold CV:
Note that between the repeat, the data is variant (not simple repeat, repeat with randomness).

In [ ]:
rkf = RepeatedKFold(n_splits=3, n_repeats=5, random_state=2020)
for train_index, test_index in rkf.split(X_data):
      print("Train:")
      print(X_data[train_index])
      print("Test:")
      print(X_data[test_index])
      print('\n')

Train:
[[3 2]
 [0 0]
 [0 0]
 [1 3]
 [0 1]
 [3 0]]
Test:
[[4 2]
 [2 3]
 [2 3]]


Train:
[[3 2]
 [4 2]
 [0 0]
 [2 3]
 [2 3]
 [3 0]]
Test:
[[0 0]
 [1 3]
 [0 1]]


Train:
[[0 0]
 [4 2]
 [2 3]
 [1 3]
 [0 1]
 [2 3]]
Test:
[[3 2]
 [0 0]
 [3 0]]


Train:
[[3 2]
 [0 0]
 [4 2]
 [0 0]
 [2 3]
 [3 0]]
Test:
[[2 3]
 [1 3]
 [0 1]]


Train:
[[3 2]
 [2 3]
 [1 3]
 [0 1]
 [2 3]
 [3 0]]
Test:
[[0 0]
 [4 2]
 [0 0]]


Train:
[[0 0]
 [4 2]
 [0 0]
 [2 3]
 [1 3]
 [0 1]]
Test:
[[3 2]
 [2 3]
 [3 0]]


Train:
[[3 2]
 [0 0]
 [2 3]
 [1 3]
 [0 1]
 [2 3]]
Test:
[[0 0]
 [4 2]
 [3 0]]


Train:
[[0 0]
 [4 2]
 [0 0]
 [1 3]
 [0 1]
 [3 0]]
Test:
[[3 2]
 [2 3]
 [2 3]]


Train:
[[3 2]
 [0 0]
 [4 2]
 [2 3]
 [2 3]
 [3 0]]
Test:
[[0 0]
 [1 3]
 [0 1]]


Train:
[[0 0]
 [4 2]
 [2 3]
 [1 3]
 [0 1]
 [2 3]]
Test:
[[3 2]
 [0 0]
 [3 0]]


Train:
[[3 2]
 [0 0]
 [0 0]
 [2 3]
 [2 3]
 [3 0]]
Test:
[[4 2]
 [1 3]
 [0 1]]


Train:
[[3 2]
 [4 2]
 [0 0]
 [1 3]
 [0 1]
 [3 0]]
Test:
[[0 0]
 [2 3]
 [2 3]]


Train:
[[4 2]
 [2 3]
 [1 3]
 [0 1]
 [2 3

### Stratified K-fold Cross Validation

Stratified K-fold CV is particularly useful when the data is imbalanced. See below code for the use of the `StratifiedKFold` method.

In [ ]:
skf = StratifiedKFold(n_splits=5, random_state=None)
# X is the feature set and y is the target
for train_index, val_index in skf.split(X,y):
    print("Train:", train_index, "Validation:", val_index)
    X_train, X_test = X[train_index], X[val_index]
    y_train, y_test = y[train_index], y[val_index]


Train: [ 53  54  56  57  62  64  65  70  72  73  75  77  78  82  83  85  86  87
  91  94  95  99 100 105 108 117 118 119 121 122 126 127 129 131 132 134
 135 138 141 146 154 155 156 157 158 159 160 161 162 163 164 165 166 167
 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221
 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239
 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257
 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275
 276 277 278 279 280 281 282 283 284 285 286 287 288 289 290 291 292 293
 294 295 296 297 298 299 300 301 302 303 304 305 306 307 308 309 310 311
 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329
 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 347
 348 349 350 351 352 353 354 355 356 357 358

As said above, for model evaluation purposes, we can simply use the `cross_val_score` function. In the `cross_val_score` function, if the `cv` value is _integer_, the model (e.g. `clf`) is a classifier, and `y` is _categorical_ (e.g. _binary_ in this case),  `StratifiedKFold` is used. In all other cases, `KFold` is used. In short, `cross_val_score` by default apply stratified K-fold CV for classification problems.

### Cross Validation for Time Series

Time series data is very special - since the time sequence is implied in the data. Thus, splitting a time-series dataset randomly does not work because the time section of your data will be messed up. For a time series forecasting problem, we perform cross validation in the forward chaining manner.

```
fold 1: training [1], test [2]
fold 2: training [1 2], test [3]
fold 3: training [1 2 3], test [4]
fold 4: training [1 2 3 4], test [5]
fold 5: training [1 2 3 4 5], test [6]
.
.
.
fold n: training [1 2 3 ….. n-1], test [n]
```

In [ ]:
tscv = TimeSeriesSplit(n_splits=3)
for train_index, test_index in tscv.split(X_data):
      print("Train:")
      print(X_data[train_index])
      print("Test:")
      print(X_data[test_index])
      print('\n')

Train:
[[3 2]
 [0 0]
 [4 2]]
Test:
[[0 0]
 [2 3]]


Train:
[[3 2]
 [0 0]
 [4 2]
 [0 0]
 [2 3]]
Test:
[[1 3]
 [0 1]]


Train:
[[3 2]
 [0 0]
 [4 2]
 [0 0]
 [2 3]
 [1 3]
 [0 1]]
Test:
[[2 3]
 [3 0]]




# Conclusion

In this tutorial, we discussed the importance of cross validation in machine learning models, in particular we focused on the __model evaluation__ use of CV. Thus, you should be comfortable using the `cross_val_score` function in your model evaluation phase.

We also surveyed the most popular CV methods - `sklearn` support most of them natively. In other projects, you may want to use other CV methods.

In next week's tutorial, we will use CV for another important purpose: model optimization. Till then, try CV on your own to evaluate your models.